In [3]:
# import lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [4]:
import pandas as pd 

df = pd.read_csv('D:/project_data_mining/src/dataset.csv')

In [5]:
def split_by_time_cat(df_cat):
    train = df_cat[df_cat["YEAR"] <= 2020]
    val   = df_cat[(df_cat["YEAR"] >= 2021) & (df_cat["YEAR"] <= 2022)]
    test  = df_cat[df_cat["YEAR"] >= 2025]
    return train, val, test


In [6]:
# metric
from sklearn.metrics import mean_absolute_error, mean_squared_error

def smape(y_true, y_pred):
    return 100 * np.mean(
        2 * np.abs(y_pred - y_true) /
        (np.abs(y_true) + np.abs(y_pred) + 1e-8)
    )

def pred_10(y_true, y_pred):
    return np.mean(
        np.abs(y_pred - y_true) / (np.abs(y_true) + 1e-8) < 0.1
    ) * 100

def evaluate(y_true, y_pred):
    return {
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "MAE": mean_absolute_error(y_true, y_pred),
        "sMAPE": smape(y_true, y_pred),
        "Pred10": pred_10(y_true, y_pred)
    }



In [7]:
# train SARIMAx 1 category 
from statsmodels.tsa.statespace.sarimax import SARIMAX

def sarimax_one_category(df_cat, order=(1,1,1), seasonal_order=(0,1,1,12)):
    train, val, test = split_by_time_cat(df_cat)

    y_train = train["CPI"]
    y_test  = test["CPI"]

    model = SARIMAX(
        y_train,
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )

    res = model.fit(disp=False)

    y_pred = res.forecast(steps=len(test))

    result = test[["TIME_PERIOD", "Expenditure_category"]].copy()
    result["CPI_actual"] = y_test.values
    result["CPI_pred_ARIMA"] = y_pred.values  # ARIMA = SARIMA không exog

    metrics = evaluate(y_test.values, y_pred.values)

    return result, metrics


In [8]:
#  train all category 
all_results = []
all_metrics = []

for cat in df["Expenditure_category"].unique():
    df_cat = df[df["Expenditure_category"] == cat].copy()

    # safety check
    if len(df_cat) < 60:
        print(f"Skip {cat} (too short)")
        continue

    result_cat, metrics_cat = sarimax_one_category(df_cat)

    all_results.append(result_cat)
    all_metrics.append({
        "Category": cat,
        **metrics_cat
    })




d:\project_data_mining\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
d:\project_data_mining\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
d:\project_data_mining\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
d:\project_data_mining\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model w

In [9]:
# gộp 
forecast_df = pd.concat(all_results, ignore_index=True)
forecast_df.tail()

,TIME_PERIOD,Expenditure_category,CPI_actual,CPI_pred_ARIMA
115,2025-06-01,Transport,101.602,75.115418
116,2025-07-01,Transport,101.822,75.452718
117,2025-08-01,Transport,102.053,75.882218
118,2025-09-01,Transport,103.679,76.005418
119,2025-10-01,Transport,104.846,76.083018


In [10]:
# results
metrics_df = pd.DataFrame(all_metrics)
metrics_df

,Category,RMSE,MAE,sMAPE,Pred10
0,All Items,14.665371,14.658702,15.691934,0.0
1,Clothing and footwear,1.748627,1.562242,1.582565,100.0
2,Communication,0.821272,0.719647,0.731712,100.0
3,Education,9.148606,9.148223,9.528577,100.0
4,Food and non-alcoholic beverages,12.056136,12.046644,12.684389,0.0
5,"Furnishings, household equipment and routine h...",4.463272,4.437108,4.554377,100.0
6,Health,12.433883,12.415574,12.924516,0.0
7,"Housing, water, electricity, gas and other fuels",14.002941,13.988048,14.953042,0.0
8,Miscellaneous goods and services,4.187365,4.173967,4.272030,100.0
9,Recreation and culture,13.818202,13.790094,15.040762,0.0


In [11]:
y_true_all = forecast_df["CPI_actual"].values
y_pred_all = forecast_df["CPI_pred_ARIMA"].values


def pred_k(y_true, y_pred, k=0.15):
    return np.mean(
        np.abs(y_pred - y_true) / (np.abs(y_true) + 1e-8) < k
    ) * 100

overall_metrics = {
    "RMSE": float(np.sqrt(mean_squared_error(y_true_all, y_pred_all))),
    "MAE": mean_absolute_error(y_true_all, y_pred_all),
    "sMAPE(%)": float(smape(y_true_all, y_pred_all)),
    "Pred10(%)": float(pred_k(y_true_all, y_pred_all, 0.1))
}

overall_metrics


{'RMSE': 12.773303672950423,
 'MAE': 10.698903488250323,
 'sMAPE(%)': 11.476615826717062,
 'Pred10(%)': 41.66666666666667}